In [1]:
import torch as pt
import torch.nn as nn
import torch.nn.functional as F

In [2]:
class SymNet(nn.Module):
    def __init__(self, Nlayers, NchOutLayers, GnnPerms, GDict, gdiags, NNSites,
                 SitesToShells, dim, act="softplus"):
        """
        :param Nlayers : no. of layers excluding the vector conv layer
               Note : the last conv layer must output a single channel.
               
        :param NchOutLayers : no. of output channels of each layer
        
        :param GnnPerms : the permutation of nearest neighbor vectors due to group operations.
                This will be used to rotate the filters.
        
        :param Gdict : contains the cartesian rotation matrix of each group operation, with
                Indices assigned to each group op. Format: {Index : g}
                Note : every "i^th" row of GnnPerms must correspond to the Group operation 
                with index "i" in GDict.
        
        :param gdiags : block diagonal matrix, containing the cartesian rotation matrices
                        of group operations in the diagonal block from the top left as given
                        by the index of the group operation in GDict
        
        :param NNSites : contains nearest neighbor indices of each site. Sites go along the 
                         columns of the first row, and nn sites of each site appear in the 
                         rows below it in the same column.
        
        :param SitesToShells : Nsite-length torch array that gives which neighbor shell a site
                               belongs to
        
        :param dim : dimensionality of cartesian vectors
        
        Note : the last layer is actually the second last layer, because
        before R3 conv, we'll add another layer that outputs a single channel.
        """
        super().__init__()
        
        self.Nlayers = Nlayers
        self.GnnPerms = GnnPerms
        self.Ng = GnnPerms.shape[0]
        self.N_ngb = GnnPerms.shape[1]
        self.NNSites = NNSites
        self.Nsites = NNSites.shape[1]
        
        if SitesToShells.shape[0] != self.Nsites:
            raise ValueError("Site to shell indexing is not correct")
        
        self.NchOutLayers = NchOutLayers
        self.GDict = GDict
        self.dim = dim
        
#         # Create the block diagonal group matrix for transforming the R3 layer filters
#         self.gdiags = pt.zeros(self.Ng*self.dim, self.Ng*self.dim).double()
#         for gInd, g in GDict.items():
#             rowStart = gInd * self.dim
#             rowEnd = (gInd + 1) * self.dim
#             self.gdiags[rowStart : rowEnd, rowStart : rowEnd] = pt.tensor(g.cartrot).double()
        
        
        if self.NchOutLayers[-1] != 1:
            raise ValueError("The last conv layer must output a single channel")
        
        self.weightList = nn.ParameterList([])
        self.biasList = nn.ParameterList([])
        # define weights first
        for layer in range(Nlayers):
            if layer == 0:
                # Layer 0 processes the input state
                NchIn = 1
            else:
                NchIn = NchOutLayers[layer-1]
            
            Layerweights = pt.normal(2, 1, size=(NchOutLayers[layer], NchIn, N_ngb),
                                     requires_grad=True).double()
            LayerBias = pt.normal(2, 1, size=(NchOutLayers[layer], 1)).double()
            
            
            self.weightList.append(nn.Parameter(Layerweights))
            self.biasList.append(nn.Parameter(LayerBias))
        
        # Now make the last vector conv layer
        self.wtVC = nn.Parameter(pt.normal(2, 1, size=(self.dim, N_ngb),
                                           requires_grad=True).double())
        
        # Make the shell parameters
        Nshell = pt.max(SitesToShells)
        
        self.ShellWeights = nn.Parameter(pt.normal(2, 1, size=(Nshells,)
                                                   ,requires_grad = True).double())
        
        self.activation = F.relu if act=="relu" else F.softplus
        
    def RotateParams(self):
        """
        Constructs symmetrically rotated weight matrices
        """
        
        self.GWeights = []
        self.Gbias = []
        
        for Idx, weight in enumerate(self.weightList):
            
            # First, get the input and output channels
            NchIn = 1 if Idx == 0 else self.NchOutLayers[Idx-1]
            NchOut = self.NchOutLayers[Idx]
            
            # First repeat the weight
            weightRepeat = weight.repeat_interleave(self.Ng, dim=0)
            
            # Then repeat the permutation indices
            GnnPerm_repeat = self.GnnPerms.repeat(NchOut, NchIn).view(-1, 
                                                                      NchIn,
                                                                      self.N_ngb)
            
            # Then gather according to the indices
            weight_repeat_perm = pt.gather(weightRepeat, 2, GnnPerm_repeat).view(-1, NchIn*self.N_ngb)
            
            # Store the repeated weights
            self.GWeights.append(weight_repeat_perm)
            
            # store the repeated biases
            bias = self.biasList[Idx]
            bias_repeat = bias.repeat_interleave(self.Ng, dim=0)
            
            self.Gbias.append(bias_repeat)
        
        # Now we need to process the vector convolution layer
        
        # First, we repeat the weights
        wtVC_repeat = self.wtVC.repeat(self.Ng, 1)
        
        # The we repeat group permutation indices
        GnnPerm_repeat = self.GnnPerms.repeat_interleave(self.dim, dim=0)            
        self.wtVC_repeat_transf = pt.matmul(self.gdiags, pt.gather(wtVC_repeat, 1, GnnPerm_repeat))
        
        # Repeat the shell indices
        self.SiteShellWeights = self.ShellWeights[SitesToShells]
    
    def RearrangeToInput(self, In):
        """
        Takes the output of a layer and rearranges it into a suitable output
        for the next layer.
        
        :param In : the tensor to rearrange.
               "In" should have the shape (N_batch x Nch x Nsites), where
               N_batch is the no. of sample states in the batch
               Nch is the no. of channels in the output
               Nsites is the no. of sites in the supercell.
               
               An additional N_ngb subchannels are added to each channel.
               So, if the incoming image had four channels, the function will
               return an image with N_ngb*4 channels. Each subchannel of a channel
               contains nearest neighbor information of each site indexed in the
               same manner as NNSites.
        """
        
        Nch = In.shape[1]
        out = In.repeat_interleave(self.N_ngb, dim=1)
        NNRepeat = self.NNSites.unsqueeze(0).repeat(out.shape[0], Nch, 1)        
        return pt.gather(out, 2, NNRepeat)
    
    def forward(self, InStates, UseShellWeights=True, Test=False):
        """
        :param InStates : input states with shape (N_batch, Nch, Nsites)
        """
        Nbatch = InStates.shape[0]
        NSites = InStates.shape[2]
        # Expand to include nearest neighbors
        out = self.RearrangeToInput(InStates)
        
        outlayers = []
        outlayersG = []
        
        InLayer = []
        
        # Now do the scalar kernel convolutions
        for layer in range(self.Nlayers):
            
            # Get the weights and biases
            Psi = self.GWeights[layer]
            bias = self.Gbias[layer]
            
            NchOut = self.NchOutLayers[layer]
            
            if Test:
                InLayer.append(out.clone().detach().data)
            
            
            # do the convolution
            out = self.activation((pt.matmul(Psi, out) + bias)).view(Nbatch, NchOut, self.Ng, NSites)
            
            if Test:
                outlayersG.append(out.clone().detach().data)
            
            # do the group averaging
            out = pt.sum(out, dim=2)/self.Ng
            
            if Test:
                outlayers.append(out.clone().detach().data)
            
            # Rearrange input for the next layer
            out = self.RearrangeToInput(out)
        
        # Finally, do the R3 convolution
        # out should now have the shape (N_batch, N_ngb, Nsites)
        out = pt.matmul(self.wtVC_repeat_transf, out).view(Nbatch, self.Ng, self.dim, NSites)
        
        # Then group average
        out = pt.sum(out, dim=1)/self.Ng
        
        if UseShellWeights:
            out = out*self.SiteShellWeights.view(out.shape[0], 1, out.shape[2])
        
        outVecSites = out.clone().data
        
        # Then site average
        out = pt.sum(out, dim=2)/NSites
        
        if Test:
            return InLayer, outlayersG, outlayers, outVecSites, out 
        
        return out